In [1]:
import pandas as pd

In [2]:
ch = pd.read_csv('data/mock/rotational_churn_mock_data.csv', dtype='str')
ch.head()

,IMEIs,IMSIs,MSISDNs,Phone_Num,Contacted_Nums,Called,Texted,call_site,text_site,contact_date,user_apps,net_hrs_spent(hrs),load_amt(php),mobile_data_usage(gb),active_time
0,219742189102274,515022095584684,639904771101,09904771101,09254175505,no,yes,NS,T599,2019-08-09,"paypal, viber, YT Music, lazada, twitch, discord",1.3,96,4.51,day
1,219742189102274,515022095584684,639904771101,09904771101,09241347050,yes,yes,T2129,T1056,2020-11-07,"paymaya, messenger, gcash, reddit, shopee, twi...",3.5,260,3.15,night
2,219742189102274,515022095584684,639904771101,09904771101,09914781099,yes,yes,T10193,T7592,2020-10-02,"chrome, twitch, discord, viber, youtube, paypa...",11.4,83,4.82,night
3,219742189102274,515022095584684,639904771101,09904771101,09538031002,yes,no,T3938,NS,2020-02-03,"discord, grab, viber, paypal, paymaya, youtube...",19.6,208,1.53,night
4,219742189102274,515022095584684,639904771101,09904771101,09251415391,yes,no,T2028,NS,2019-09-27,"discord, chrome, YT Music, netflix, spotify, y...",7.1,347,2.24,night


### I am taking the average number of hrs each person has spent on the internet, their average load amt, and their average mobile data

In [3]:
ch['net_hrs_spent(hrs)'] = ch['net_hrs_spent(hrs)'].astype(float)
ch['load_amt(php)'] = ch['net_hrs_spent(hrs)'].astype(float)
ch['mobile_data_usage(gb)'] = ch['net_hrs_spent(hrs)'].astype(float)
df = ch.drop(columns=['contact_date']).copy()

#, 'IMSIs', 'MSISDNs', 'Phone_Num'
df = df.groupby(['IMEIs'], as_index=False, sort=False).mean()
df.head()

# I still have no idea how I am going to compare each user on whether they are a churned user... 

,IMEIs,net_hrs_spent(hrs),load_amt(php),mobile_data_usage(gb)
0,219742189102274,9.718750,9.718750,9.718750
1,811838545143465,11.200000,11.200000,11.200000
2,763919762969531,10.612500,10.612500,10.612500
3,267455025264981,10.527778,10.527778,10.527778
4,406065995086603,12.671429,12.671429,12.671429


In [4]:
len(df)

501

In [5]:
# I want to test something.. (probably stupid; update: IT'S REALLY STUPID)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import numpy as np

In [6]:
x = df.loc[:, 'net_hrs_spent(hrs)':'mobile_data_usage(gb)'].copy()
y = df.loc[:,'IMEIs'].copy()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.8, random_state=0)

In [7]:
nn = KNeighborsClassifier()
nn.fit(x_train,y_train)

KNeighborsClassifier()

In [8]:
pred = nn.predict(x_test)

result = pd.DataFrame({'Old':pred,'New':y_test})
result.head()

,Old,New
90,335888215172077,448602439603682
254,198572622421253,200650971121934
284,324148569740851,962917315710476
446,351244418269261,7807596397972710
339,324148569740851,218223062420837


What I need to do now is to compute the euclidean distance of every user compared to other users in the dataframe.... yep.... That will give my machine a lot of strain. (Might do it in kaggle or elsewhere if that is the case)

In [27]:
print(ch.groupby('IMEIs')['Contacted_Nums'].max().head())
ch.groupby('IMEIs')['Contacted_Nums','Called'].max().head()

IMEIs
101901440889712     09966553017
103784008957578     09999434466
104226383620636     09942673099
104915693516811     09592372916
1089939570579510    09998832859
Name: Contacted_Nums, dtype: object
<ipython-input-27-efbbcf1f04f7>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  ch.groupby('IMEIs')['Contacted_Nums','Called'].max().head()


,Contacted_Nums,Called
IMEIs,,
101901440889712,09966553017,yes
103784008957578,09999434466,yes
104226383620636,09942673099,yes
104915693516811,09592372916,yes
1089939570579510,09998832859,yes


#### Progress So Far:
 - Created mock data that will be the base data pattern for the churners which will be implemented later on.
 - Researched on what to look out for when dealing with rotational churners. Idiro fingerprinting is an option that will be considered in this project in order to look at users that have similar patterns in contacts. (bit difficult to simulate) Another method would be to analyze the behavior of each user up until they churned through time-series analysis, and check data from the time the user churned onwards if there are other data with similar behavior patterns as the person that churned.

#### Progress needed later on: (still related to data cleaning)
 - Simulate churners based on mock data - similarity in contacts, similarity in user activity and behavior, similarity in promos used etc.
 - **Figure out a way to view the data as a time series data** - the reason for this is to not only figure out globally if there is a trend of churning at a certain point, it is also to view the behavior of the users individually before they became rotational churners or as they are about to become churners. It is also to view the data orderly.
 - Need to fix script on creating mock data to fit the statement above.
 - Measure the similarity of 2 users based on contact tracing/idiro fingerprinting and/or behavior.
 - Get some sleep.

#### Other progress needed:
 - I need to have a column called "number of times called" which would show how many times a person has been called.
 - I also need to find a way to show the most frequent person called in this notebook.